In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import re
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K



In [2]:
import requests
from datetime import date, timedelta
from datetime import datetime as dt
import pytest
import os
# from api_call_examples import *
import json
import pandas as pd
import numpy as np
import sqlalchemy as db
import datetime

#### Initial Exploration

In [4]:
data=pd.read_csv("./2021/n2au2101.csv",skiprows=4).drop([0,1,2,3])

In [5]:
data.head(10)

,# Data type,Code,Year,Week,Day,Date(dd.mm.yyyy),Alias,Unit,Hour1,Hour2,...,Hour16,Hour17,Hour18,Hour19,Hour20,Hour21,Hour22,Hour23,Hour24,Avg/Total
4,PR,AO,2021,1.0,1.0,04.01.2021,UK,GBP,56.98,56.97,...,60.06,60.11,73.92,100.40,83.98,72.50,60.03,57.06,54.72,60.70
5,TU,AB,2021,1.0,1.0,04.01.2021,UK,MWh,5422.80,4991.80,...,8447.60,9131.30,8881.80,10011.90,9534.80,8971.90,7007.60,6356.60,6914.40,180879.20
6,TU,AS,2021,1.0,1.0,04.01.2021,UK,MWh,5422.80,4991.80,...,8447.60,9131.30,8881.80,10011.90,9534.80,8971.90,7007.60,6356.60,6914.40,180879.20
7,PR,AO,2021,1.0,2.0,05.01.2021,UK,GBP,55.95,54.01,...,57.92,57.44,77.72,103.75,88.00,74.00,57.00,53.99,48.95,60.88
8,TU,AB,2021,1.0,2.0,05.01.2021,UK,MWh,5193.50,5161.10,...,9577.70,10923.90,9975.40,10570.90,9924.20,9696.60,8768.10,8180.70,8673.90,197089.50
9,TU,AS,2021,1.0,2.0,05.01.2021,UK,MWh,5193.50,5161.10,...,9577.70,10923.90,9975.40,10570.90,9924.20,9696.60,8768.10,8180.70,8673.90,197089.50
10,PR,AO,2021,1.0,3.0,06.01.2021,UK,GBP,46.49,46.41,...,90.39,75.00,563.04,1000.04,383.28,156.74,80.30,55.06,51.06,145.08
11,TU,AB,2021,1.0,3.0,06.01.2021,UK,MWh,8176.10,7721.40,...,10461.30,10035.80,9301.20,9543.30,9986.20,10607.00,9118.20,8470.50,8859.30,223287.10
12,TU,AS,2021,1.0,3.0,06.01.2021,UK,MWh,8176.10,7721.40,...,10461.30,10035.80,9301.20,9543.30,9986.20,10607.00,9118.20,8470.50,8859.30,223287.10
13,PR,AO,2021,1.0,4.0,07.01.2021,UK,GBP,55.02,52.90,...,81.09,78.96,120.08,489.60,159.92,87.41,67.95,57.18,52.01,91.80


In [ ]:
price=data[data["Unit"]=="GBP"]

In [ ]:
price

In [ ]:
price_melt=pd.melt(
    price.drop(["# Data type","Code","Avg/Total","Hour3B"],axis=1),
    id_vars=["Year","Week","Day","Date(dd.mm.yyyy)","Alias","Unit"],
    var_name="hours",
    value_name="price"
    

)

In [ ]:
price_melt.head(24)


#### Extraction to Pandas DataFrame

In [3]:
folders=os.listdir("./")

In [4]:

# Put all data into a single dataframe

match_count=0
full_df=pd.DataFrame()

for folder in folders:
    
    if re.match('20*',folder):  
        
        folder_path="./"+folder
        
        csvlist=os.listdir(folder_path)
        
        for file_count,file in enumerate(csvlist):
            
            # print("lookin at ", file)
            file_path=folder_path+"/"+file
            
            temp_df=pd.read_csv(file_path,skiprows=4).drop([0,1,2,3]).reset_index()
            
            temp_df=temp_df.drop(["# Data type","Code","Avg/Total"],axis=1)
            
            if file_count==0:
                temp_full_df=temp_df
            else:
                temp_full_df=temp_full_df.append(temp_df,ignore_index=True)
                
        if match_count==0:
            full_df=temp_full_df
            
            match_count+=1
            
        else:
            print(full_df.shape)
            
            full_df=full_df.append(temp_full_df,ignore_index=True)
            


(1116, 31)
(2260, 31)
(3404, 31)
(4548, 31)
(6019, 33)
(7556, 33)
(9127, 33)
(10999, 33)
(12871, 33)
(14743, 33)
(16648, 33)


In [5]:
full_df1=full_df.drop(["index"],axis=1).dropna(how="all") # drop rows where all elements are nan

In [6]:
# replace NaN values from Hour3 with values from Hour3A amd Hour3B
full_df2=full_df1
full_df2["Hour3"]= full_df1["Hour3"].fillna(value=full_df1["Hour3A"])

full_df2["Hour3"]=full_df1["Hour3"].fillna(value=full_df1["Hour3B"])


full_df2=full_df2.dropna(subset=["Hour3"])

In [7]:
full_df2["Hour3"].isna().sum() #now all the na rows should be gone

0

In [8]:
full_df3=full_df2.drop(["Hour3A","Hour3B"],axis=1) # 3A and 3B no longer necessary since all moved to 3

In [9]:
price=full_df3[full_df3["Unit"]=="GBP"] # create a separate table for price

In [10]:
# Melting the table- making it so that all hours are in rows instead of columns
price_melt=pd.melt(
    price.drop(["Alias","Unit"],axis=1),
    id_vars=["Year","Week","Day","Date(dd.mm.yyyy)"],
    var_name="hours",
    value_name="price(GBP)"
    

)

In [11]:
price_final=price_melt.drop_duplicates(subset=["Year","Week","Day","hours"]).reset_index(drop=True) # Getting rid of duplicate data

In [12]:
units=full_df3[full_df3["Unit"]=="MWh"]

In [13]:
units_melt=pd.melt(
    units.drop(["Alias", "Unit"],axis=1),
    id_vars=["Year","Week","Day","Date(dd.mm.yyyy)"],
    var_name="hours",
    value_name="Units(MWh)"
    

)

In [14]:
units_final=units_melt.drop_duplicates(subset=["Year","Week","Day","hours"]).reset_index(drop=True)

In [15]:
units_final["hours"]= units_final.apply(lambda x: x["hours"].strip("Hour"),axis=1)

units_final["hours"]=units_final["hours"].astype(int)

In [16]:
units_final["Date(dd.mm.yyyy)"]=pd.to_datetime(units_final["Date(dd.mm.yyyy)"],dayfirst=True)
price_final["Date(dd.mm.yyyy)"]=pd.to_datetime(price_final["Date(dd.mm.yyyy)"],dayfirst=True)



In [17]:
def doy(x):
    x=pd.Period(x,freq="D")
    x=x.dayofyear
    return x

units_final["Date(dd.mm.yyyy)"]=units_final["Date(dd.mm.yyyy)"].map(doy)

In [18]:
units_final

,Year,Week,Day,Date(dd.mm.yyyy),hours,Units(MWh)
0,2010,2.0,3.0,13,1,253.0
1,2010,2.0,4.0,14,1,253.0
2,2010,2.0,5.0,15,1,227.0
3,2010,2.0,6.0,16,1,146.0
4,2010,2.0,7.0,17,1,160.0
...,...,...,...,...,...,...
98491,2021,15.0,5.0,106,24,6540.2
98492,2021,15.0,6.0,107,24,6224.8
98493,2021,15.0,7.0,108,24,6697.0
98494,2021,16.0,1.0,109,24,5981.3


In [25]:
print((units_final["Units(MWh)"]).max())
print((units_final["Units(MWh)"]).min())
print(units_final["Units(MWh)"].mean())
print(units_final["Units(MWh)"].std())

24586.0
0.0
10582.1585546622
5326.604177900157


### Model

In [27]:
X=units_final

In [28]:
X["Year"]=X["Year"].astype(int)

X=X.rename(columns={"Date(dd.mm.yyyy)": "Day of Year"})
X["Day of Year"]=X["Day of Year"].astype(int)

In [29]:
y=price_final["price(GBP)"]

In [53]:
X.columns

Index(['Year', 'Week', 'Day', 'Day of Year', 'hours', 'Units(MWh)'], dtype='object')

In [30]:
scaler=StandardScaler()
std_X=scaler.fit_transform(X)

In [31]:
pickle_scaler="../../reporter/reporter_app/trading/model/pickle_scaler.pkl"

pickle.dump(scaler, open(pickle_scaler,'wb'))



In [55]:
std_X.mean(axis=0)

array([ 1.15093842e-14,  1.34395419e-16, -7.40982793e-17,  1.06838226e-16,
        2.48159013e-17, -1.38435217e-16])

In [56]:
np.isnan(std_X[:,5])

array([False, False, False, ..., False, False, False])

In [57]:
types=[type(X[i][10]) for i in X.columns]
print(types)

  


[<class 'numpy.int32'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.int32'>, <class 'numpy.int32'>, <class 'numpy.float64'>]


In [58]:
x_train, x_test, y_train, y_test = train_test_split(std_X, y, test_size=0.2, random_state=40)

In [29]:
print(x_train.shape,x_test.shape, y_train.shape, y_test.shape)

(78796, 6) (19700, 6) (78796,) (19700,)


In [60]:
model=MLPRegressor((365,52,100))
model.fit(x_train,y_train)

MLPRegressor(hidden_layer_sizes=(365, 52, 100))

In [62]:
# Save to file in the current working directory
pkl_filename = "../../reporter/reporter_app/trading/model/pickle_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)


In [63]:

# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)
    
y_pred=pickle_model.predict(x_test)

In [67]:
y_pred -y_test

55650    -5.088545
34507     7.234717
51334     5.433792
41711    -5.166230
93316   -14.840848
           ...    
4347     -0.323055
4249      0.134937
38226    -4.228292
46556     2.844977
48231     4.695026
Name: price(GBP), Length: 19700, dtype: float64

In [30]:
model = keras.Sequential(
    [
        layers.Dense(3650,activation="relu",name="Rodney",input_shape=(6,)),
        layers.Dense(365,activation="sigmoid",name="Rooney"),
        layers.Dense(1,name="layra")
    ]
)

model.compile(optimizer="adam", loss="mean_squared_error",metrics=["mean_squared_error"])

In [31]:

model.fit(x_train,y_train,epochs=3,validation_data=(x_test,y_test),batch_size=2000,shuffle=False)
model.save_weights("../../reporter/reporter_app/trading/saved_model.h5")

Epoch 1/3


InternalError:  Blas GEMM launch failed : a.shape=(2000, 6), b.shape=(6, 3650), m=2000, n=3650, k=6
	 [[node sequential/Rodney/MatMul (defined at <ipython-input-31-6b9fc7b9597d>:1) ]] [Op:__inference_train_function_639]

Function call stack:
train_function


In [1]:
model.load_weights("../../reporter/reporter_app/trading/saved_model.h5")

NameError: name 'model' is not defined

In [6]:
processed_tab=pd.DataFrame({"Year":[2021,2021], "Week":[1,36],"Day":[5,7],"Day of Year":[4,255],"hours":[2,18],"Units(MWh)":[23345,34456]})

In [31]:
y_pred=model.predict(x_test)

InternalError:  Blas GEMM launch failed : a.shape=(32, 6), b.shape=(6, 3650), m=32, n=3650, k=6
	 [[node sequential/Rodney/MatMul (defined at <ipython-input-31-77e3652b0a4e>:1) ]] [Op:__inference_predict_function_181]

Function call stack:
predict_function


In [ ]:
y_pred.shape

In [ ]:
pred=np.array(y_pred)

In [ ]:
pred.shape

In [ ]:
test=np.array(y_test)
test=test[:,np.newaxis]

In [ ]:
test.shape

In [ ]:
test.shape==(19700,1)

In [ ]:
print(type(y_pred),type(test))

In [ ]:
difference=y_pred-test

In [ ]:
difference.shape


In [ ]:
plt.plot(difference)

In [ ]:
def call_model(L1=64,L2=8,L3=4,L4=2, batch=400 , epoch=200):
    # Define Sequential model with 4 hidden layers, 
    model = keras.Sequential(
        [
            layers.Dense(L1, activation="sigmoid", name="layer1", input_shape=(6,)),
            layers.Dense(L2, activation="sigmoid", name="layer2"),
            layers.Dense(L3, activation="sigmoid", name="layer3"),
            layers.Dense(L4, activation="sigmoid", name="layer4"),
            layers.Dense(1,name="layer5"),
        ]
    )
    model.compile(optimizer='adam', loss='mean_squared_error',metrics=['mean_squared_error'])
    print (model.summary())
    model.fit(x_train,y_train,epochs=epoch, validation_data=(x_test,y_test),batch_size=batch, shuffle=False)
    model.save_weights("./saved_model")
    return model

In [ ]:
call_model(L1=256,L2=2000,L3=200,L4=8,batch=8000, epoch=300)


The model is clearly lacking! It might be better to improve how it is modelling. Before that we need to test if the model can be loaded and predictions made with new data.

In [ ]:
model=keras.models.load_model("./saved_model")

In [ ]:
y_pred=model.predict(x_test)

In [ ]:
y_pred

This doesn't work at all but its not as important as being able to call the model. The question to answer is where are we going to save the model- what would a solution look like?

In [ ]:
oil_data=pd.read_csv("https://www.quandl.com/api/v3/datasets/OPEC/ORB.csv?api_key=Another api key from Quandl")

In [ ]:
oil_data.head()


In [ ]:
bp_eGen=pd.read_csv("https://www.quandl.com/api/v3/datasets/BP/ELEC_GEN_GBR.csv?api_key=Another api key from Quandl")

In [ ]:
# Yearly generation
bp_eGen

In [ ]:
plt.scatter(X["Units(MWh)"],y)
plt.xlabel("Units")
plt.ylabel("Price(GBP)")

In [ ]:
y.mean()

In [ ]:
y.std()

In [ ]:
y.max()

In [ ]:
y.argmax()

In [ ]:
X[78106:78120]

In [ ]:
def get_predicted_load_next_day():
    api_key= "Please request your api key"
    
    base_url="https://api.bmreports.com"
    tdelta=timedelta(days=1)
    settlementdate=(datetime.date.today()+tdelta).isoformat()
    tab=pd.read_csv(f"{base_url}/BMRS/B0620/V1?ServiceType=CSV&Period=*&APIKey={api_key}&SettlementDate={settlementdate}",skiprows=4)
    filtered_tab=tab[["Settlement Date", "Settlement Period", "Quantity"]]
    filtered_tab=filtered_tab.dropna(subset=["Settlement Date"])
    
    return filtered_tab

In [ ]:
api_key= "Please request your api key"
tdelta=timedelta(days=1)
settlementdate=(date.today()).isoformat()
    
base_url="https://api.bmreports.com"
response=requests.get(f"https://api.bmreports.com/BMRS/B0620/V1?ServiceType=CSV&Period=*&APIKey={api_key}&SettlementDate={settlementdate}")

In [ ]:
len(response.text)

In [ ]:
print(response.text)


In [ ]:
print(settlementdate)

In [ ]:
filtered_tab= get_predicted_load_next_day()

In [ ]:
filtered_tab

In [ ]:
# Ceil division to get the hour
filtered_tab["Hour"]=[-(-x//2) for x in filtered_tab["Settlement Period"]]

In [ ]:
filtered_tab.head()

In [ ]:
def get_wday_wk_doy(x):
    x=datetime.date.fromisoformat(x)
    year=x.isocalendar()[0]
    week=x.isocalendar()[1]
    wday=x.isocalendar()[2]    
    doy=x.timetuple().tm_yday
    return year,week,wday,doy

In [ ]:
filtered_tab["week"],filtered_tab["wday"],filtered_tab["doy"]=zip(*filtered_tab["Settlement Date"].map(get_wday_wk_doy))

In [ ]:
filtered_tab.head()

In [ ]:
type(filtered_tab["Settlement Date"][1])



In [ ]:
print([x for x in filtered_tab["Settlement Date"]])

In [ ]:
filtered_tab["Year"]=[int(x[0:4]) for x in filtered_tab["Settlement Date"]]
filtered_tab["Month"]=[int(x[5:7]) for x in filtered_tab["Settlement Date"]]
filtered_tab["Day"]=[int(x[8:10]) for x in filtered_tab["Settlement Date"]]

In [ ]:
tab=pd.read_csv(f"{base_url}/BMRS/B0620/V1?ServiceType=CSV&Period=*&APIKey={api_key}&SettlementDate={settlementdate}",skiprows=4)
tab